<a href="https://colab.research.google.com/github/cekiorpes/big-data-challenge/blob/main/Level_2/Merge_reviews_ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("ReviewCloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
# Load in review data for Video Games into a DataFrame
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

video_game_df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Video_Games_v1_00.tsv.gz"), inferSchema=True, sep='\t', header=True, timestampFormat="yyyy-mm-dd")
video_game_df.show(10)

In [ ]:
# Load in review data for Pet Products into a DataFrame
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Pet_Products_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

pet_df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Pet_Products_v1_00.tsv.gz"), inferSchema=True, sep='\t', header=True, timestampFormat="yyyy-mm-dd")
pet_df.show(10)

In [ ]:
#Merge DataFrames
joined_df= user_data_df.join(user_payment_df, how="outer")
joined_df.show()

In [ ]:
print(joined_df.count())
joined_df = joined_df.dropna()
print(joined_df.count())

In [ ]:
joined_df.printSchema()

In [ ]:
#Create new DataFrame for review_id_table
review_id_table = joined_df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
review_id_table.show()

In [ ]:
#Create new DataFrame for products table
products = joined_df.select(["product_id", "product_title"])
products.show()

In [ ]:
#Drop duplicates
products = products.dropDuplicates()

In [ ]:
#Create new DataFrame for customers table
customers = joined_df.select(["customer_id"])
customers.show(5)

In [ ]:
customers = joined_df.groupby("customer_id").agg({"customer_id":"count"})
customers.show(5)

In [ ]:
#Rename count column
customers = customers.withColumnRenamed("count(customer_id)", "customer_count")
customers.show(5)

In [ ]:
#Create new DataFrame for vine_table
vine_table = joined_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_table.show()

In [ ]:
#Write DataFrame to RDS
mode="append"
jdbc_url = "jdbc:postgresql://mypostgresdb.cojhn4lpsels.us-east-1.rds.amazonaws.com:5432/big_data_db"
config = {"user":"root",
          "password": "",
          "driver":"org.postgresql.Driver"}

In [ ]:
#Write review_id_table DataFrame to RDS
review_id_table.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
#Write products DataFrame to RDS
products.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
#Write customers DataFrame to RDS
customers.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [ ]:
#Write vine_table DataFrame to RDS
vine_table.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)